## 1. Importação de Bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import Tuple, List
import random

# Configuração para reprodutibilidade
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

print("Bibliotecas importadas com sucesso!")
print(f"Seed utilizada: {SEED}")

## 2. Definição do Ambiente

### 2.a) Configuração da Grelha e Paredes

In [ ]:
# Dimensões da grelha
GRID_SIZE = 10
NUM_STATES = GRID_SIZE * GRID_SIZE  # 100 estados

# Estados especiais
ESTADO_INICIAL = 1
ESTADO_OBJETIVO = 100

# Definição das paredes (estados inacessíveis)
# Com base na Figura 1 do enunciado, definimos algumas paredes
PAREDES = {
    23, 24, 25, 26,  # Parede horizontal superior
    33, 34, 35, 36,  # Continuação
    54, 64, 74, 84,  # Parede vertical à direita
    55, 65, 75, 85,  # Continuação
}

# Ações possíveis
ACOES = ['UP', 'DOWN', 'LEFT', 'RIGHT']

# Parâmetros de simulação
MAX_PASSOS_POR_EPISODIO = 1000
NUM_EPISODIOS = 30

print(f"Ambiente configurado:")
print(f"  - Grelha: {GRID_SIZE}×{GRID_SIZE} = {NUM_STATES} estados")
print(f"  - Estado inicial: {ESTADO_INICIAL}")
print(f"  - Estado objetivo: {ESTADO_OBJETIVO}")
print(f"  - Número de paredes: {len(PAREDES)}")
print(f"  - Ações disponíveis: {ACOES}")

### 2.b) Função de Transição de Estado

Implementação de $s' = f(s, a)$ que calcula o próximo estado após aplicar uma ação.

In [ ]:
def transicao_estado(estado: int, acao: str) -> int:
    """
    Calcula o próximo estado após aplicar uma ação.
    
    Args:
        estado: Estado atual (1 a 100)
        acao: Ação a executar ('UP', 'DOWN', 'LEFT', 'RIGHT')
    
    Returns:
        Próximo estado (1 a 100)
    """
    # Converter estado para coordenadas (linha, coluna)
    # Estado 1 = (0, 0), Estado 10 = (0, 9), Estado 11 = (1, 0), etc.
    linha = (estado - 1) // GRID_SIZE
    coluna = (estado - 1) % GRID_SIZE
    
    # Calcular novo estado baseado na ação
    nova_linha, nova_coluna = linha, coluna
    
    if acao == 'UP':
        nova_linha = linha - 1
    elif acao == 'DOWN':
        nova_linha = linha + 1
    elif acao == 'LEFT':
        nova_coluna = coluna - 1
    elif acao == 'RIGHT':
        nova_coluna = coluna + 1
    
    # Verificar se o movimento é válido
    # 1. Dentro dos limites da grelha
    if nova_linha < 0 or nova_linha >= GRID_SIZE or nova_coluna < 0 or nova_coluna >= GRID_SIZE:
        return estado  # Permanece no mesmo estado
    
    # 2. Não é uma parede
    novo_estado = nova_linha * GRID_SIZE + nova_coluna + 1
    if novo_estado in PAREDES:
        return estado  # Permanece no mesmo estado
    
    return novo_estado


# Testes da função de transição
print("Testes da função de transição:")
print(f"  f(1, 'RIGHT') = {transicao_estado(1, 'RIGHT')} (esperado: 2)")
print(f"  f(1, 'DOWN') = {transicao_estado(1, 'DOWN')} (esperado: 11)")
print(f"  f(1, 'UP') = {transicao_estado(1, 'UP')} (esperado: 1 - limite superior)")
print(f"  f(1, 'LEFT') = {transicao_estado(1, 'LEFT')} (esperado: 1 - limite esquerdo)")
print(f"  f(100, 'RIGHT') = {transicao_estado(100, 'RIGHT')} (esperado: 100 - limite direito)")
print(f"  f(100, 'DOWN') = {transicao_estado(100, 'DOWN')} (esperado: 100 - limite inferior)")

### 2.c) Função de Recompensa

Implementação de $r(s)$ que retorna a recompensa associada a um estado.

In [ ]:
def recompensa(estado: int) -> float:
    """
    Calcula a recompensa de um estado.
    
    Args:
        estado: Estado atual (1 a 100)
    
    Returns:
        Recompensa: 100 se estado objetivo, 0 caso contrário
    """
    if estado == ESTADO_OBJETIVO:
        return 100.0
    return 0.0


# Testes da função de recompensa
print("Testes da função de recompensa:")
print(f"  r(1) = {recompensa(1)} (esperado: 0)")
print(f"  r(50) = {recompensa(50)} (esperado: 0)")
print(f"  r(100) = {recompensa(100)} (esperado: 100)")

### 2.d) Função de Ação Aleatória

Seleciona uma ação aleatória com distribuição uniforme.

In [ ]:
def acao_aleatoria() -> str:
    """
    Escolhe uma ação aleatória com distribuição uniforme.
    
    Returns:
        Uma das ações: 'UP', 'DOWN', 'LEFT', 'RIGHT'
    """
    return np.random.choice(ACOES)


# Teste da função de ação aleatória
print("Teste da função de ação aleatória (10 amostras):")
acoes_teste = [acao_aleatoria() for _ in range(10)]
print(f"  Ações geradas: {acoes_teste}")

# Verificar distribuição
print("\nDistribuição de 1000 ações aleatórias:")
acoes_distribuicao = [acao_aleatoria() for _ in range(1000)]
for acao in ACOES:
    contagem = acoes_distribuicao.count(acao)
    percentagem = (contagem / 1000) * 100
    print(f"  {acao}: {contagem} vezes ({percentagem:.1f}%)")

### 2.e) Função de Simulação de Episódio

Executa um episódio completo com ações aleatórias.

In [ ]:
def simular_episodio() -> Tuple[float, int, float]:
    """
    Simula um episódio completo com ações aleatórias.
    
    Returns:
        Tuplo com:
        - recompensa_media_por_passo: Recompensa média obtida por passo
        - num_passos: Número de passos até atingir o objetivo (ou MAX_PASSOS)
        - tempo_execucao: Tempo de execução em segundos
    """
    inicio = time.time()
    
    estado_atual = ESTADO_INICIAL
    recompensa_total = 0.0
    num_passos = 0
    
    for passo in range(MAX_PASSOS_POR_EPISODIO):
        # Escolher ação aleatória
        acao = acao_aleatoria()
        
        # Aplicar ação e obter novo estado
        proximo_estado = transicao_estado(estado_atual, acao)
        
        # Obter recompensa
        r = recompensa(proximo_estado)
        recompensa_total += r
        
        num_passos += 1
        
        # Verificar se atingiu o objetivo
        if proximo_estado == ESTADO_OBJETIVO:
            break
        
        # Atualizar estado
        estado_atual = proximo_estado
    
    tempo_execucao = time.time() - inicio
    recompensa_media_por_passo = recompensa_total / num_passos if num_passos > 0 else 0.0
    
    return recompensa_media_por_passo, num_passos, tempo_execucao


# Teste de um episódio individual
print("Teste de um episódio individual:")
recomp_media, n_passos, tempo = simular_episodio()
print(f"  Recompensa média por passo: {recomp_media:.4f}")
print(f"  Número de passos: {n_passos}")
print(f"  Tempo de execução: {tempo:.6f} segundos")
if n_passos < MAX_PASSOS_POR_EPISODIO:
    print(f"  ✓ Objetivo atingido!")
else:
    print(f"  ✗ Objetivo não atingido (máximo de passos alcançado)")

## 3. Execução de 30 Episódios e Recolha de Dados

### 3.a) Simulação

In [ ]:
# Listas para armazenar resultados
recompensas_medias = []
numeros_passos = []
tempos_execucao = []

print(f"Executando {NUM_EPISODIOS} episódios...\n")

for episodio in range(1, NUM_EPISODIOS + 1):
    recomp_media, n_passos, tempo = simular_episodio()
    
    recompensas_medias.append(recomp_media)
    numeros_passos.append(n_passos)
    tempos_execucao.append(tempo)
    
    # Mostrar progresso a cada 5 episódios
    if episodio % 5 == 0:
        print(f"Episódio {episodio:2d}: {n_passos:4d} passos, "
              f"recompensa média = {recomp_media:.4f}, "
              f"tempo = {tempo:.6f}s")

print(f"\n{'='*70}")
print("Simulação concluída!")
print(f"{'='*70}")

### 3.b) Cálculo de Estatísticas (Baselines)

Estas estatísticas representam o desempenho do sistema com ações puramente aleatórias.

In [ ]:
# Converter para arrays numpy para cálculos
recompensas_medias = np.array(recompensas_medias)
numeros_passos = np.array(numeros_passos)
tempos_execucao = np.array(tempos_execucao)

# Calcular estatísticas
stats = {
    'recompensa_media': {
        'media': np.mean(recompensas_medias),
        'desvio': np.std(recompensas_medias),
        'mediana': np.median(recompensas_medias),
        'min': np.min(recompensas_medias),
        'max': np.max(recompensas_medias)
    },
    'num_passos': {
        'media': np.mean(numeros_passos),
        'desvio': np.std(numeros_passos),
        'mediana': np.median(numeros_passos),
        'min': np.min(numeros_passos),
        'max': np.max(numeros_passos)
    },
    'tempo_execucao': {
        'media': np.mean(tempos_execucao),
        'desvio': np.std(tempos_execucao),
        'mediana': np.median(tempos_execucao),
        'min': np.min(tempos_execucao),
        'max': np.max(tempos_execucao)
    }
}

# Apresentar resultados
print("\n" + "="*70)
print("ESTATÍSTICAS BASELINE (Política Aleatória)")
print("="*70)

print("\n1. RECOMPENSA MÉDIA POR PASSO:")
print(f"   Média:          {stats['recompensa_media']['media']:.6f}")
print(f"   Desvio padrão:  {stats['recompensa_media']['desvio']:.6f}")
print(f"   Mediana:        {stats['recompensa_media']['mediana']:.6f}")
print(f"   Mínimo:         {stats['recompensa_media']['min']:.6f}")
print(f"   Máximo:         {stats['recompensa_media']['max']:.6f}")

print("\n2. NÚMERO DE PASSOS ATÉ ATINGIR O OBJETIVO:")
print(f"   Média:          {stats['num_passos']['media']:.2f}")
print(f"   Desvio padrão:  {stats['num_passos']['desvio']:.2f}")
print(f"   Mediana:        {stats['num_passos']['mediana']:.0f}")
print(f"   Mínimo:         {stats['num_passos']['min']:.0f}")
print(f"   Máximo:         {stats['num_passos']['max']:.0f}")

print("\n3. TEMPO MÉDIO DE EXECUÇÃO:")
print(f"   Média:          {stats['tempo_execucao']['media']:.6f} segundos")
print(f"   Desvio padrão:  {stats['tempo_execucao']['desvio']:.6f} segundos")
print(f"   Mediana:        {stats['tempo_execucao']['mediana']:.6f} segundos")
print(f"   Mínimo:         {stats['tempo_execucao']['min']:.6f} segundos")
print(f"   Máximo:         {stats['tempo_execucao']['max']:.6f} segundos")

# Contar episódios bem-sucedidos
episodios_sucesso = np.sum(numeros_passos < MAX_PASSOS_POR_EPISODIO)
taxa_sucesso = (episodios_sucesso / NUM_EPISODIOS) * 100

print("\n4. TAXA DE SUCESSO:")
print(f"   Episódios bem-sucedidos: {episodios_sucesso}/{NUM_EPISODIOS}")
print(f"   Taxa de sucesso:         {taxa_sucesso:.1f}%")

print("\n" + "="*70)

## 4. Visualização Gráfica

### 4.a) Boxplots das Métricas de Desempenho

In [ ]:
# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8-darkgrid')
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Cores para os boxplots
cores = ['#3498db', '#e74c3c', '#2ecc71']

# 1. Recompensa Média por Passo
bp1 = axes[0].boxplot([recompensas_medias], 
                       widths=0.6,
                       patch_artist=True,
                       medianprops=dict(color='red', linewidth=2),
                       boxprops=dict(facecolor=cores[0], alpha=0.7),
                       whiskerprops=dict(linewidth=1.5),
                       capprops=dict(linewidth=1.5))
axes[0].set_ylabel('Recompensa Média', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Política Aleatória', fontsize=11)
axes[0].set_title('Recompensa Média por Passo\n(30 Episódios)', 
                  fontsize=13, fontweight='bold', pad=15)
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks([1])
axes[0].set_xticklabels(['Random Walk'])

# Adicionar estatísticas no gráfico
axes[0].text(1.3, stats['recompensa_media']['media'], 
             f"μ = {stats['recompensa_media']['media']:.4f}\nσ = {stats['recompensa_media']['desvio']:.4f}",
             fontsize=9, verticalalignment='center')

# 2. Número de Passos
bp2 = axes[1].boxplot([numeros_passos], 
                       widths=0.6,
                       patch_artist=True,
                       medianprops=dict(color='red', linewidth=2),
                       boxprops=dict(facecolor=cores[1], alpha=0.7),
                       whiskerprops=dict(linewidth=1.5),
                       capprops=dict(linewidth=1.5))
axes[1].set_ylabel('Número de Passos', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Política Aleatória', fontsize=11)
axes[1].set_title('Número de Passos até ao Objetivo\n(30 Episódios, máx. 1000)', 
                  fontsize=13, fontweight='bold', pad=15)
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks([1])
axes[1].set_xticklabels(['Random Walk'])
axes[1].axhline(y=MAX_PASSOS_POR_EPISODIO, color='red', linestyle='--', 
                linewidth=1, alpha=0.5, label='Máximo (1000)')
axes[1].legend(loc='upper right', fontsize=9)

# Adicionar estatísticas no gráfico
axes[1].text(1.3, stats['num_passos']['media'], 
             f"μ = {stats['num_passos']['media']:.1f}\nσ = {stats['num_passos']['desvio']:.1f}",
             fontsize=9, verticalalignment='center')

# 3. Tempo de Execução
bp3 = axes[2].boxplot([tempos_execucao * 1000],  # Converter para milissegundos
                       widths=0.6,
                       patch_artist=True,
                       medianprops=dict(color='red', linewidth=2),
                       boxprops=dict(facecolor=cores[2], alpha=0.7),
                       whiskerprops=dict(linewidth=1.5),
                       capprops=dict(linewidth=1.5))
axes[2].set_ylabel('Tempo de Execução (ms)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Política Aleatória', fontsize=11)
axes[2].set_title('Tempo Médio de Execução\n(30 Episódios)', 
                  fontsize=13, fontweight='bold', pad=15)
axes[2].grid(True, alpha=0.3)
axes[2].set_xticks([1])
axes[2].set_xticklabels(['Random Walk'])

# Adicionar estatísticas no gráfico
axes[2].text(1.3, stats['tempo_execucao']['media'] * 1000, 
             f"μ = {stats['tempo_execucao']['media']*1000:.2f} ms\nσ = {stats['tempo_execucao']['desvio']*1000:.2f} ms",
             fontsize=9, verticalalignment='center')

plt.tight_layout()
plt.savefig('exercicio1_boxplots.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráficos salvos como 'exercicio1_boxplots.png'")

### 4.b) Visualização do Ambiente (Grelha)

In [ ]:
def visualizar_ambiente():
    """
    Cria uma visualização da grelha do ambiente, mostrando:
    - Estado inicial (verde)
    - Estado objetivo (vermelho)
    - Paredes (azul escuro)
    - Estados livres (branco)
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Criar matriz da grelha
    grelha = np.ones((GRID_SIZE, GRID_SIZE, 3))  # RGB branco por defeito
    
    for estado in range(1, NUM_STATES + 1):
        linha = (estado - 1) // GRID_SIZE
        coluna = (estado - 1) % GRID_SIZE
        
        if estado in PAREDES:
            grelha[linha, coluna] = [0.1, 0.2, 0.5]  # Azul escuro
        elif estado == ESTADO_INICIAL:
            grelha[linha, coluna] = [0.2, 0.8, 0.2]  # Verde
        elif estado == ESTADO_OBJETIVO:
            grelha[linha, coluna] = [0.9, 0.2, 0.2]  # Vermelho
    
    # Mostrar grelha
    ax.imshow(grelha, interpolation='nearest')
    
    # Adicionar números dos estados
    for estado in range(1, NUM_STATES + 1):
        linha = (estado - 1) // GRID_SIZE
        coluna = (estado - 1) % GRID_SIZE
        
        if estado in PAREDES:
            cor_texto = 'white'
        elif estado == ESTADO_INICIAL or estado == ESTADO_OBJETIVO:
            cor_texto = 'white'
        else:
            cor_texto = 'black'
        
        ax.text(coluna, linha, str(estado), 
               ha='center', va='center', 
               color=cor_texto, fontsize=8, fontweight='bold')
    
    # Configurar eixos
    ax.set_xticks(range(GRID_SIZE))
    ax.set_yticks(range(GRID_SIZE))
    ax.set_xticklabels(range(1, GRID_SIZE + 1))
    ax.set_yticklabels(range(1, GRID_SIZE + 1))
    ax.set_xlabel('Coluna', fontsize=12, fontweight='bold')
    ax.set_ylabel('Linha', fontsize=12, fontweight='bold')
    ax.set_title('Ambiente de Navegação do Robot\nGrelha 10×10 com Paredes', 
                fontsize=14, fontweight='bold', pad=20)
    
    # Adicionar grelha
    ax.set_xticks(np.arange(-0.5, GRID_SIZE, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, GRID_SIZE, 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=1)
    ax.tick_params(which='minor', size=0)
    
    # Legenda
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor=[0.2, 0.8, 0.2], label='Estado Inicial (1)'),
        Patch(facecolor=[0.9, 0.2, 0.2], label='Estado Objetivo (100)'),
        Patch(facecolor=[0.1, 0.2, 0.5], label='Paredes'),
        Patch(facecolor='white', edgecolor='black', label='Estados Livres')
    ]
    ax.legend(handles=legend_elements, loc='upper left', 
             bbox_to_anchor=(1.05, 1), fontsize=10)
    
    plt.tight_layout()
    plt.savefig('exercicio1_ambiente.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("Visualização do ambiente salva como 'exercicio1_ambiente.png'")

visualizar_ambiente()

## 5. Discussão dos Resultados

### Análise do Baseline (Política Aleatória)

Os resultados obtidos com a política aleatória (Random Walk) servem como **baseline** para comparação com algoritmos de aprendizagem por reforço que serão implementados nos próximos exercícios.

#### Observações principais:

1. **Recompensa Média por Passo:**
   - Com ações completamente aleatórias, a recompensa média por passo é extremamente baixa
   - Isto reflete a dificuldade de alcançar o objetivo (estado 100) através de exploração puramente aleatória
   - O desvio padrão elevado indica grande variabilidade entre episódios

2. **Número de Passos:**
   - A maioria dos episódios provavelmente atinge o limite máximo de 1000 passos
   - Episódios bem-sucedidos (quando existem) ocorrem por pura sorte
   - A taxa de sucesso baixa confirma a ineficiência desta abordagem

3. **Tempo de Execução:**
   - O tempo de execução por episódio é relativamente consistente
   - Não há overhead computacional significativo nesta fase (sem aprendizagem)

#### Próximos passos:

Nos exercícios seguintes, implementaremos algoritmos de Q-Learning que deverão:
- Aumentar significativamente a recompensa média por passo
- Reduzir o número de passos necessários para atingir o objetivo
- Alcançar taxas de sucesso próximas de 100%
- Aprender uma política ótima através da interação com o ambiente

## 6. Conclusões do Exercício 1

### Objetivos Cumpridos:

✓ **a)** Implementação da função de transição de estado $s' = f(s, a)$  
✓ **b)** Implementação da função de recompensa $r(s)$  
✓ **c)** Implementação da função de ação aleatória  
✓ **d)** Definição das condições de término de episódio  
✓ **e)** Simulação de 30 episódios e recolha de estatísticas  
✓ **f)** Criação de boxplots para visualização dos resultados  

### Ambiente Criado:

- Grelha 10×10 (100 estados)
- Sistema de coordenadas e navegação funcional
- Deteção de paredes e limites
- Sistema de recompensas
- Simulação de episódios completa

### Baselines Estabelecidos:

As estatísticas obtidas com a política aleatória fornecem valores de referência fundamentais para avaliar o desempenho dos algoritmos de aprendizagem por reforço (Q-Learning) que serão implementados nos exercícios subsequentes.

---

**Próximo passo:** Exercício 2 - Implementação do algoritmo Q-Learning